##Perfect Matchings in Cubic Bridgeless Graphs
####Auxiliary Code

###imports and constants

In [1]:
!pip install scipy
!pip install --upgrade scipy

In [2]:
from scipy.optimize import linprog

import itertools
from copy import deepcopy
from collections import defaultdict

In [3]:
epsilon = 10 ** (-12)

###paths

In [4]:
non_burl_paths_of_length_9 = [
    [[1, 2], [2, 3], [1, 2], [2, 3], [1, 2], [2, 3], [1, 2], [2, 3], [1, 2]]
]

non_burl_paths_of_length_8 = [
    [[1, 2], [2, 3], [1, 2], [2, 3], [1, 2], [2, 3], [1, 2], [2, 3]],
    [[1, 2], [2, 3], [1, 2], [2, 3], [1, 3], [1, 2], [1, 3], [1, 2]],
    [[1, 2], [2, 3], [1, 3], [2, 3], [1, 2], [1, 3], [1, 2], [2, 3]],
]

non_burl_paths_of_length_7 = [
    [[1, 2], [2, 3], [1, 2], [2, 3], [1, 2], [2, 3], [1, 2]],
    [[1, 2], [2, 3], [1, 2], [2, 3], [1, 3], [1, 2], [1, 3]],
    [[1, 2], [2, 3], [1, 2], [1, 3], [2, 3], [1, 3], [2, 3]],
    [[1, 2], [2, 3], [1, 2], [2, 3], [1, 3], [2, 3], [1, 3]],
    [[1, 2], [2, 3], [1, 2], [1, 3], [2, 3], [1, 3], [1, 2]],
    [[1, 2], [2, 3], [1, 3], [2, 3], [1, 2], [1, 3], [1, 2]],
    [[1, 2], [2, 3], [1, 3], [1, 2], [1, 3], [2, 3], [1, 2]],
    [[1, 2], [2, 3], [1, 3], [2, 3], [1, 3], [1, 2], [2, 3]],
    [[1, 2], [2, 3], [1, 3], [1, 2], [1, 3], [1, 2], [2, 3]],
]

non_burl_paths_of_length_6 = [
    [[1, 2], [2, 3], [1, 2], [2, 3], [1, 2], [2, 3]],
    [[1, 2], [2, 3], [1, 2], [2, 3], [1, 3], [1, 2]],
    [[1, 2], [2, 3], [1, 2], [2, 3], [1, 3], [2, 3]],
    [[1, 2], [2, 3], [1, 3], [1, 2], [1, 3], [1, 2]],
    [[1, 2], [2, 3], [1, 2], [1, 3], [1, 2], [1, 3]],
    [[1, 2], [2, 3], [1, 2], [1, 3], [2, 3], [1, 3]],
    [[1, 2], [2, 3], [1, 3], [1, 2], [1, 3], [2, 3]],
    [[1, 2], [2, 3], [1, 3], [2, 3], [1, 2], [1, 3]],
    [[1, 2], [2, 3], [1, 3], [2, 3], [1, 3], [1, 2]],
]

non_burl_paths_of_length_5 = [
    [[1, 2], [2, 3], [1, 2], [2, 3], [1, 2]],
    [[1, 2], [2, 3], [1, 2], [2, 3], [1, 3]],
    [[1, 2], [2, 3], [1, 3], [2, 3], [1, 3]],
    [[1, 2], [2, 3], [1, 2], [1, 3], [1, 2]],
    [[1, 2], [2, 3], [1, 2], [1, 3], [2, 3]],
    [[1, 2], [2, 3], [1, 3], [1, 2], [1, 3]],
    [[1, 2], [2, 3], [1, 3], [2, 3], [1, 2]]
]

non_burl_paths_of_length_4 = [
    [[1, 2], [2, 3], [1, 2], [2, 3]],
    [[1, 2], [2, 3], [1, 2], [1, 3]],
    [[1, 2], [2, 3], [1, 3], [2, 3]],
    [[1, 2], [2, 3], [1, 3], [1, 2]]
]

non_burl_paths_of_length_3 = [
    [[1, 2], [2, 3], [1, 2]],
    [[1, 2], [2, 3], [1, 3]]
]

non_burl_paths_of_length_2 = [
    [[1, 2], [2, 3]]
]

In [5]:
burl_paths_of_length_5 = [
    [[1, 2], [2, 3], [1, 3], [1, 2], [2, 3]]
]

burl_paths_of_length_6 = [
    [[1, 2], [2, 3], [1, 2], [1, 3], [1, 2], [2, 3]],
    [[1, 2], [2, 3], [1, 3], [2, 3], [1, 3], [2, 3]],
]

burl_paths_of_length_7 = [
    [[1, 2], [2, 3], [1, 3], [2, 3], [1, 3], [1, 2], [1, 3]],
]

burl_paths_of_length_8 = [
    [[1, 2], [2, 3], [1, 2], [2, 3], [1, 3], [1, 2], [1, 3], [2, 3]],
    [[1, 2], [2, 3], [1, 3], [1, 2], [1, 3], [2, 3], [1, 2], [2, 3]],
]

burl_paths_of_length_10 = [
    [[1, 2], [2, 3], [1, 2], [2, 3], [1, 2], [2, 3], [1, 2], [2, 3], [1, 2], [2, 3]]
]

###to_graph

In [6]:
def first_vertex_at_pos(pos):
    return (pos + 1) * 2 - 1

def second_vertex_at_pos(pos):
    return (pos + 1) * 2

def add_edge(graph, v1, v2):
    if v1 >= 0: graph[v1].add(v2)
    if v2 >= 0: graph[v2].add(v1)

def path_to_graph(path):
    number_of_vertices = len(path) * 2

    graph = dict()
    for v in range(1, number_of_vertices + 1):
        graph[v] = set()

    for i in range(len(path)):

        v1 = path[i][0]
        v2 = path[i][1]

        v_first = i * 2 + 1
        v_second = i * 2 + 2

        if i < len(path) - 1:
            if v1 == path[i + 1][0]:
                add_edge(graph, v_first, first_vertex_at_pos(i + 1))
            elif v1 == path[i + 1][1]:
                add_edge(graph, v_first, second_vertex_at_pos(i + 1))
            elif (i < len(path) - 2) and (v1 == path[i + 2][0]):
                add_edge(graph, v_first, first_vertex_at_pos(i + 2))
            elif (i < len(path) - 2) and (v1 == path[i + 2][1]):
                add_edge(graph, v_first, second_vertex_at_pos(i + 2))

            if v2 == path[i + 1][0]:
                add_edge(graph, v_second, first_vertex_at_pos(i + 1))
            elif v2 == path[i + 1][1]:
                add_edge(graph, v_second, second_vertex_at_pos(i + 1))
            elif (i < len(path) - 2) and (v2 == path[i + 2][0]):
                add_edge(graph, v_second, first_vertex_at_pos(i + 2))
            elif (i < len(path) - 2) and (v2 == path[i + 2][1]):
                add_edge(graph, v_second, second_vertex_at_pos(i + 2))

        add_edge(graph, v_first, v_second)
    return graph

def is_cubic(graph):
    count_deg = defaultdict(int)
    for v_from in graph:
        for v_to in graph[v_from]:
            count_deg[v_to] += 1
            if v_to < 0:
                count_deg[v_from] += 1
    for vertex in count_deg:
        if vertex >= 0 and count_deg[vertex] != 3:
            print(vertex)
            print(graph)
            return False
    return True

###adding_ends

In [7]:
def add_b3_left(graph):
    pairs = [[1, 2], [1, 4], [2, 4]]
    new_graphs = []
    for pair in pairs:
        new_graph = dict()
        new_graph[0] = set()
        for vertex in graph: new_graph[vertex] = deepcopy(graph[vertex])

        add_edge(new_graph, 0, pair[0])
        add_edge(new_graph, 0, pair[1])

        add_edge(new_graph, 0, -1)
        add_edge(new_graph, 7 - pair[0] - pair[1], -2)

        new_graphs.append(deepcopy(new_graph))
    return new_graphs

def add_b3_right(graph):
    vertices = sorted(list(graph.keys()))
    new_vertex = vertices[-1] + 1

    new_graphs = []
    for i in range(1, 4):
        new_graph = dict()
        for vertex in graph: new_graph[vertex] = deepcopy(graph[vertex])
        new_graph[new_vertex] = set()

        v = len(vertices) - 1
        number_of_vertex_of_deg_2 = 0
        while number_of_vertex_of_deg_2 < 3:
            if len(graph[vertices[v]]) <= 2:
                number_of_vertex_of_deg_2 += 1
                if number_of_vertex_of_deg_2 == i:
                    add_edge(new_graph, vertices[v], -4)
                else:
                    add_edge(new_graph, vertices[v], new_vertex)
            v -= 1

        add_edge(new_graph, new_vertex, -5)

        new_graphs.append(deepcopy(new_graph))
    return new_graphs

def add_b3_both_ends(graph):
    result = []
    for new_graph in add_b3_left(graph):
        result += deepcopy(add_b3_right(new_graph))
    return result

In [8]:
def add_k4_left(graph):
    new_graph = dict()
    new_graph[1] = set([-1, 3, 4])
    new_graph[2] = set([-2, 3, 5])
    new_graph[3] = set([1, 2, 7])
    for v in graph:
        new_v = v + 3
        new_graph[new_v] = set()
        for v_to in graph[v]:
            new_graph[new_v].add(v_to + 3)
        if new_v == 4:
            new_graph[new_v].add(1)
        elif new_v == 5:
            new_graph[new_v].add(2)
        elif new_v == 7:
            new_graph[new_v].add(3)
    return new_graph

def add_k4_right(graph):
    vs = sorted(graph)
    vs[-1]
    right = [vs[-1] + 1, vs[-1] + 2, vs[-1] + 3]
    new_graph = dict()
    r1 = set([-right[0], right[2]])
    r2 = set([-right[1], right[2]])
    r3 = set([right[0], right[1]])
    for v in graph:
        new_graph[v] = deepcopy(graph[v])
        if (len(graph[v]) < 3):
            if (v == vs[-3]) or (v == vs[-4]):
                new_graph[v].add(right[2])
                r3.add(v)
            elif v == vs[-2]:
                new_graph[v].add(right[0])
                r1.add(v)
            elif v == vs[-1]:
                new_graph[v].add(right[1])
                r2.add(v)
    new_graph[right[0]] = r1
    new_graph[right[1]] = r2
    new_graph[right[2]] = r3

    return new_graph

def add_k4_both_ends(graph):
    new_graph = add_k4_right(add_k4_left(graph))
    if is_cubic(new_graph):
        return new_graph
    else:
        raise Exception(f"Not Cubic: {new_graph}")

In [9]:
def add_empty_ends_left(g):
    graph = deepcopy(g)
    graph[1].add(-1)
    graph[2].add(-2)
    graph[4].add(-4)
    return graph

def add_empty_ends_right(g):
    graph = deepcopy(g)
    vertices = sorted(graph)
    i = len(vertices) - 1
    count = 0
    while count < 3:
        if len(graph[vertices[i]]) < 3:
            if -vertices[i] not in graph[vertices[i]]:
                graph[vertices[i]].add(-vertices[i])
            else:
                graph[vertices[i]].add(-100 -vertices[i])
            count += 1
        i -= 1
    return graph

def add_empty_both_sides(graph):
    graph = add_empty_ends_right(add_empty_ends_left(graph))
    if is_cubic(graph):
        return graph
    else:
        raise Exception(f"not cubic: {graph}")

###finding_perfect_matchings

In [10]:
def matchings_in_graph(graph, covered):
    matchings = []
    for v_from in graph:
        if v_from not in covered:
            covered.add(v_from)
            for v_to in graph[v_from]:
                if v_to not in covered:
                    new_covered = deepcopy(covered)
                    new_covered.add(v_to)
                    matchings.append([[v_from, v_to]])

                    for matching in matchings_in_graph(graph, new_covered):
                        matchings.append([[v_from, v_to]] + matching)
            return matchings
    return []

def is_perfect(graph, matching):
    vertices = list(graph.keys())

    covered_vertices = set()
    for edge in matching:
        v1 = edge[0]
        v2 = edge[1]
        if ((v1 in covered_vertices) and (v1 >= 0)) or ((v2 in covered_vertices) and (v2 >= 0)):
            print("NOT A MATCHING:\n", matching)
            return False
        covered_vertices.add(v1)
        covered_vertices.add(v2)
    for vertex in vertices:
        if vertex not in covered_vertices:
            return False
    return True



def perfect_matchings(graph):
    pm = []
    for m in matchings_in_graph(graph, set()):
        if is_perfect(graph, m):
            pm.append(m)
    return pm


###alternating_cycles

In [11]:
def in_matching(edge, matching):
    if (edge in matching) or ([edge[1], edge[0]] in matching):
        return 1
    else:
        return -1

def edge_in_cycle(edge, cycle):
    return (edge in cycle) or ([edge[1], edge[0]] in cycle)

def vertex_in_cycle(vertex, cycle):
    for edge in cycle:
        if vertex in edge:
            return True
    return False

def is_alternating(cycle, matching):
    prev = 0
    for i in range(1, len(cycle)):
        edge = [cycle[i - 1], cycle[i]]
        cur = in_matching(edge, matching)
        if cur == prev:
            return False
        else:
            prev = cur
    return True

def find_cycles(graph):
    cycles = []
    visited = set()

    def dfs(node, path):
        #print(path)
        visited.add(node)
        for neighbor in graph[node]:
            if ((neighbor not in path) or (neighbor == path[0])) and (neighbor in graph):
                if neighbor in visited:
                    cycles.append(path + [neighbor])
                else:
                    dfs(neighbor, path + [neighbor])
        visited.remove(node)

    for node in graph:
        dfs(node, [node])

    return cycles

def find_alternating_cycles(graph, matching):
    cycles = find_cycles(graph)
    starting_vertices_covered = set()
    alternating = []
    for c in cycles:
        if (len(c) > 3) and (len(c) % 2 == 1):
            if is_alternating(c, matching):
                starting_vertices_covered.add(c[0])
                add = True
                for i in range(1, len(c) - 1):
                    if c[i] in starting_vertices_covered:
                        add = False
                if c[::-1] in alternating:
                    add = False
                if add:
                    alternating.append(c)
    return alternating

def is_disjoint(c1, c2):
    set1 = set(c1)
    set2 = set(c2)
    for v in set2:
        if v in set1: return False
    return True

def possible_add(cycle, to_cycles):
    for c in to_cycles:
        if not is_disjoint(c, cycle): return False
    return True

def maximum_disjoint_alternating_cycles(max, C):
    if len(C) == 0: return max
    if possible_add(C[0], max):
        v1 = maximum_disjoint_alternating_cycles(max+[C[0]], C[1:])
        v2 = maximum_disjoint_alternating_cycles(max, C[1:])
        if len(v1) > len(v2): return v1
        else: return v2
    else: return maximum_disjoint_alternating_cycles(max, C[1:])

def get_largest_set_of_disjoint_alternating_cycles(graph, matching):
    all_alternating_cycles = find_alternating_cycles(graph, matching)
    return maximum_disjoint_alternating_cycles([], all_alternating_cycles)

###LP

In [12]:
def unique_edges(graph):
    edges = []
    for v_from in graph:
        for v_to in graph[v_from]:
            edge = [v_from, v_to]
            rev_edge = [v_to, v_from]
            if (edge not in edges) and (rev_edge not in edges):
                edges.append(edge)
    return edges

In [13]:
def is_burl(graph):
    M = perfect_matchings(graph)
    number_of_x = len(M)
    c = []
    for m in M:
        cy_set = get_largest_set_of_disjoint_alternating_cycles(graph, m)
        c.append(len(cy_set))

    bounds = [(0, 1) for i in range(len(M))]

    v1 = [1 for i in range(len(M))] #sum of all probabilities
    A_eq = []
    A_eq.append(v1)
    for edge in unique_edges(graph):
        row = []
        for m in M:
            if in_matching(edge, m) == 1:
                row.append(1)
            else:
                row.append(0)
        A_eq.append(row)

    b_eq = [1/3 for i in range(len(A_eq))]
    b_eq[0] = 1 #sum of all probabilities equals 1

    res = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds)
    if res.success:
        if (res.fun >= (1/3 - epsilon)):
            print("Graph is a burl, minimum expectancy:", round(res.fun, 9))
            return True
        else:
            print("Graph is not a burl, minimum expectancy:", round(res.fun, 9))
            return False
    else:
        raise ValueError(f"Does Not Compute, matrix A: {A_eq}")

###checking_graphs

####4-cycle

In [14]:
graph = {
    1 : [-1, 2, 3],
    2 : [-2, 1, 4],
    3 : [-3, 1, 4],
    4 : [-4, 2, 3]
}
print(graph)
print("Graph is cubic:", is_cubic(graph))
print("Unique edges:", unique_edges(graph))
for m in perfect_matchings(graph):
    print("\n--------------------\n")
    d_alt_cycles_set = get_largest_set_of_disjoint_alternating_cycles(graph, m)
    d_alt_cycles_count = len(d_alt_cycles_set)
    print("Matching:", m)
    print("Number of disjoint alternating cycles:", d_alt_cycles_count)
    if d_alt_cycles_count:
        print("Cycles:")
        for c in d_alt_cycles_set:
            print(c)

{1: [-1, 2, 3], 2: [-2, 1, 4], 3: [-3, 1, 4], 4: [-4, 2, 3]}
Graph is cubic: True
Unique edges: [[1, -1], [1, 2], [1, 3], [2, -2], [2, 4], [3, -3], [3, 4], [4, -4]]

--------------------

Matching: [[1, -1], [2, -2], [3, -3], [4, -4]]
Number of disjoint alternating cycles: 0

--------------------

Matching: [[1, -1], [2, -2], [3, 4]]
Number of disjoint alternating cycles: 0

--------------------

Matching: [[1, -1], [2, 4], [3, -3]]
Number of disjoint alternating cycles: 0

--------------------

Matching: [[1, 2], [3, -3], [4, -4]]
Number of disjoint alternating cycles: 0

--------------------

Matching: [[1, 2], [3, 4]]
Number of disjoint alternating cycles: 1
Cycles:
[1, 2, 4, 3, 1]

--------------------

Matching: [[1, 3], [2, -2], [4, -4]]
Number of disjoint alternating cycles: 0

--------------------

Matching: [[1, 3], [2, 4]]
Number of disjoint alternating cycles: 1
Cycles:
[1, 2, 4, 3, 1]


In [15]:
is_burl(graph)

Graph is a burl, minimum expectancy: 0.333333333


True

####checking regular paths

#####not burls

In [16]:
for path in non_burl_paths_of_length_2:
    graph = add_empty_both_sides(path_to_graph(path))
    if is_burl(graph):
        print(graph)

Graph is not a burl, minimum expectancy: 0.0


In [17]:
for path in non_burl_paths_of_length_3:
    graph = add_empty_both_sides(path_to_graph(path))
    if is_burl(graph):
        print(graph)

Graph is not a burl, minimum expectancy: 0.0
Graph is not a burl, minimum expectancy: 0.0


In [18]:
for path in non_burl_paths_of_length_4:
    graph = add_empty_both_sides(path_to_graph(path))
    if is_burl(graph):
        print(graph)

Graph is not a burl, minimum expectancy: 0.0
Graph is not a burl, minimum expectancy: 0.0
Graph is not a burl, minimum expectancy: 0.0
Graph is not a burl, minimum expectancy: 0.0


In [19]:
for path in non_burl_paths_of_length_5:
    graph = add_empty_both_sides(path_to_graph(path))
    if is_burl(graph):
        print(graph)

Graph is not a burl, minimum expectancy: 0.0
Graph is not a burl, minimum expectancy: 0.0
Graph is not a burl, minimum expectancy: 0.0
Graph is not a burl, minimum expectancy: 0.0
Graph is not a burl, minimum expectancy: 0.0
Graph is not a burl, minimum expectancy: 0.0
Graph is not a burl, minimum expectancy: 0.0


In [20]:
for path in non_burl_paths_of_length_6:
    graph = add_empty_both_sides(path_to_graph(path))
    if is_burl(graph):
        print(graph)

Graph is not a burl, minimum expectancy: 0.0
Graph is not a burl, minimum expectancy: 0.0
Graph is not a burl, minimum expectancy: 0.0
Graph is not a burl, minimum expectancy: 0.0
Graph is not a burl, minimum expectancy: 0.0
Graph is not a burl, minimum expectancy: 0.0
Graph is not a burl, minimum expectancy: 0.0
Graph is not a burl, minimum expectancy: 0.0
Graph is not a burl, minimum expectancy: 0.0


In [21]:
for path in non_burl_paths_of_length_7:
    graph = add_empty_both_sides(path_to_graph(path))
    if is_burl(graph):
        print(graph)

Graph is not a burl, minimum expectancy: 0.0
Graph is not a burl, minimum expectancy: 0.0
Graph is not a burl, minimum expectancy: 0.0
Graph is not a burl, minimum expectancy: 0.166666667
Graph is not a burl, minimum expectancy: 0.0
Graph is not a burl, minimum expectancy: 0.0
Graph is not a burl, minimum expectancy: 0.0
Graph is not a burl, minimum expectancy: 0.0
Graph is not a burl, minimum expectancy: 0.0


In [22]:
for path in non_burl_paths_of_length_8:
    graph = add_empty_both_sides(path_to_graph(path))
    if is_burl(graph):
        print(graph)

Graph is not a burl, minimum expectancy: 0.0
Graph is not a burl, minimum expectancy: 0.0
Graph is not a burl, minimum expectancy: 0.0


In [23]:
for path in non_burl_paths_of_length_9:
    graph = add_empty_both_sides(path_to_graph(path))
    if is_burl(graph):
        print(graph)

Graph is not a burl, minimum expectancy: 0.0


#####burls

In [24]:
for path in burl_paths_of_length_5:
    graph = add_empty_both_sides(path_to_graph(path))
    if not is_burl(graph):
        print("Graph:\n", graph, "\n")

Graph is a burl, minimum expectancy: 0.333333333


In [25]:
for path in burl_paths_of_length_6:
    graph = add_empty_both_sides(path_to_graph(path))
    if not is_burl(graph):
        print("Graph:\n", graph, "\n")

Graph is a burl, minimum expectancy: 0.333333333
Graph is a burl, minimum expectancy: 0.333333333


In [26]:
for path in burl_paths_of_length_7:
    graph = add_empty_both_sides(path_to_graph(path))
    if not is_burl(graph):
        print("Graph:\n", graph, "\n")

Graph is a burl, minimum expectancy: 0.333333333


In [27]:
for path in burl_paths_of_length_8:
    graph = add_empty_both_sides(path_to_graph(path))
    if not is_burl(graph):
        print("Graph:\n", graph, "\n")

Graph is a burl, minimum expectancy: 0.333333333
Graph is a burl, minimum expectancy: 0.333333333


In [28]:
for path in burl_paths_of_length_10:
    graph = add_empty_both_sides(path_to_graph(path))
    if not is_burl(graph):
        print("Graph:\n", graph, "\n")

Graph is a burl, minimum expectancy: 0.333333333


####paths with 2-edge-cuts

In [29]:
print("\nPaths of lenth 7 with vertices on both ends such that hub of G at those vertices is isomorphic to K4")
for path in non_burl_paths_of_length_5:
    graph = add_k4_both_ends(path_to_graph(path))
    if not is_burl(graph):
        print("Graph:\n", graph, "\n")

print("\nPaths of lenth 6 with vertices on both ends such that hub of G at those vertices is isomorphic to K4")
for path in non_burl_paths_of_length_4:
    graph = add_k4_both_ends(path_to_graph(path))
    if not is_burl(graph):
        print("Graph:\n", graph, "\n")

print("\nPaths of lenth 5 with vertices on both ends such that hub of G at those vertices is isomorphic to K4")
for path in non_burl_paths_of_length_3:
    graph = add_k4_both_ends(path_to_graph(path))
    if not is_burl(graph):
        print("Graph:\n", graph, "\n")

print("\nPaths of lenth 4 with vertices on both ends such that hub of G at those vertices is isomorphic to K4")
for path in non_burl_paths_of_length_2:
    graph = add_k4_both_ends(path_to_graph(path))
    if not is_burl(graph):
        print("Graph:\n", graph, "\n")


Paths of lenth 7 with vertices on both ends such that hub of G at those vertices is isomorphic to K4
Graph is a burl, minimum expectancy: 1.0
Graph is a burl, minimum expectancy: 0.833333333
Graph is a burl, minimum expectancy: 0.833333333
Graph is a burl, minimum expectancy: 1.0
Graph is a burl, minimum expectancy: 0.833333333
Graph is a burl, minimum expectancy: 0.833333333
Graph is a burl, minimum expectancy: 0.666666667

Paths of lenth 6 with vertices on both ends such that hub of G at those vertices is isomorphic to K4
Graph is a burl, minimum expectancy: 0.666666667
Graph is a burl, minimum expectancy: 0.833333333
Graph is a burl, minimum expectancy: 0.833333333
Graph is a burl, minimum expectancy: 1.0

Paths of lenth 5 with vertices on both ends such that hub of G at those vertices is isomorphic to K4
Graph is a burl, minimum expectancy: 1.0
Graph is a burl, minimum expectancy: 0.666666667

Paths of lenth 4 with vertices on both ends such that hub of G at those vertices is isom

In [30]:
print("\nPaths of lenth 7 with vertices on both ends such that hub of G at those vertices is isomorphic to K4")
for path in non_burl_paths_of_length_5:
    for graph in add_b3_both_ends(path_to_graph(path)):
        if not is_burl(graph):
            print("Graph:\n", graph, "\n")

print("\nPaths of lenth 6 with vertices on both ends such that hub of G at those vertices is isomorphic to K4")
for path in non_burl_paths_of_length_4:
    for graph in add_b3_both_ends(path_to_graph(path)):
        if not is_burl(graph):
            print("Graph:\n", graph, "\n")

print("\nPaths of lenth 5 with vertices on both ends such that hub of G at those vertices is isomorphic to K4")
for path in non_burl_paths_of_length_3:
    for graph in add_b3_both_ends(path_to_graph(path)):
        if not is_burl(graph):
            print("Graph:\n", graph, "\n")

print("\nPaths of lenth 4 with vertices on both ends such that hub of G at those vertices is isomorphic to K4")
for path in non_burl_paths_of_length_2:
    for graph in add_b3_both_ends(path_to_graph(path)):
        if not is_burl(graph):
            print("Graph:\n", graph, "\n")


Paths of lenth 7 with vertices on both ends such that hub of G at those vertices is isomorphic to K4
Graph is a burl, minimum expectancy: 0.666666667
Graph is a burl, minimum expectancy: 1.0
Graph is a burl, minimum expectancy: 0.333333333
Graph is a burl, minimum expectancy: 1.0
Graph is a burl, minimum expectancy: 0.333333333
Graph is a burl, minimum expectancy: 0.666666667
Graph is a burl, minimum expectancy: 0.333333333
Graph is a burl, minimum expectancy: 0.666666667
Graph is a burl, minimum expectancy: 1.0
Graph is a burl, minimum expectancy: 0.666666667
Graph is a burl, minimum expectancy: 0.666666667
Graph is a burl, minimum expectancy: 1.0
Graph is a burl, minimum expectancy: 1.0
Graph is a burl, minimum expectancy: 0.666666667
Graph is a burl, minimum expectancy: 0.666666667
Graph is a burl, minimum expectancy: 0.333333333
Graph is a burl, minimum expectancy: 1.0
Graph is a burl, minimum expectancy: 0.666666667
Graph is a burl, minimum expectancy: 0.666666667
Graph is a burl

In [31]:
for path in non_burl_paths_of_length_5:
    graph = add_empty_ends_left(add_k4_right(path_to_graph(path)))
    is_cubic(graph)
    is_burl(graph)

    graph = add_empty_ends_right(add_k4_left(path_to_graph(path)))
    is_cubic(graph)
    is_burl(graph)

    for graph in add_b3_left(path_to_graph(path)):
        graph = add_empty_ends_right(graph)
        is_cubic(graph)
        if not is_burl(graph):
            print(graph)

    for graph in add_b3_right(path_to_graph(path)):
        graph = add_empty_ends_left(graph)
        is_cubic(graph)
        if not is_burl(graph):
            print(graph)

Graph is a burl, minimum expectancy: 0.666666667
Graph is a burl, minimum expectancy: 0.666666667
Graph is a burl, minimum expectancy: 0.333333333
Graph is a burl, minimum expectancy: 0.333333333
Graph is a burl, minimum expectancy: 0.333333333
Graph is a burl, minimum expectancy: 0.333333333
Graph is a burl, minimum expectancy: 0.333333333
Graph is a burl, minimum expectancy: 0.333333333
Graph is a burl, minimum expectancy: 0.666666667
Graph is a burl, minimum expectancy: 0.333333333
Graph is a burl, minimum expectancy: 0.333333333
Graph is a burl, minimum expectancy: 0.333333333
Graph is a burl, minimum expectancy: 0.333333333
Graph is a burl, minimum expectancy: 0.5
Graph is a burl, minimum expectancy: 0.666666667
Graph is a burl, minimum expectancy: 0.333333333
Graph is a burl, minimum expectancy: 0.333333333
Graph is a burl, minimum expectancy: 0.666666667
Graph is a burl, minimum expectancy: 0.333333333
Graph is a burl, minimum expectancy: 0.333333333
Graph is a burl, minimum exp

In [32]:
for path in non_burl_paths_of_length_6:
    graph = add_empty_ends_left(add_k4_right(path_to_graph(path)))
    is_cubic(graph)
    is_burl(graph)

    graph = add_empty_ends_right(add_k4_left(path_to_graph(path)))
    is_cubic(graph)
    is_burl(graph)

    for graph in add_b3_left(path_to_graph(path)):
        graph = add_empty_ends_right(graph)
        is_cubic(graph)
        if not is_burl(graph):
            print(graph)

    for graph in add_b3_right(path_to_graph(path)):
        graph = add_empty_ends_left(graph)
        is_cubic(graph)
        if not is_burl(graph):
            print(graph)

Graph is a burl, minimum expectancy: 0.666666667
Graph is a burl, minimum expectancy: 0.666666667
Graph is a burl, minimum expectancy: 0.333333333
Graph is a burl, minimum expectancy: 0.666666667
Graph is a burl, minimum expectancy: 0.333333333
Graph is a burl, minimum expectancy: 0.666666667
Graph is a burl, minimum expectancy: 0.666666667
Graph is a burl, minimum expectancy: 0.333333333
Graph is a burl, minimum expectancy: 0.666666667
Graph is a burl, minimum expectancy: 0.333333333
Graph is a burl, minimum expectancy: 0.333333333
Graph is a burl, minimum expectancy: 0.333333333
Graph is a burl, minimum expectancy: 0.666666667
Graph is a burl, minimum expectancy: 0.666666667
Graph is a burl, minimum expectancy: 0.666666667
Graph is a burl, minimum expectancy: 0.666666667
Graph is a burl, minimum expectancy: 0.777777778
Graph is a burl, minimum expectancy: 0.666666667
Graph is a burl, minimum expectancy: 0.333333333
Graph is a burl, minimum expectancy: 0.666666667
Graph is a burl, min